In [ ]:
using Oceananigans 
using Oceananigans.Units
using Oceananigans.TurbulenceClosures
using Oceananigans.Architectures: GPU
using CUDA: CuArray
using CairoMakie 
using NCDatasets
using Printf

In [2]:
# Model parameters
Nx = 300
Nz = 80
f = 1e-4               # Coriolis frequency [s⁻¹]
L_front = 10kilometers  # Initial front width [m]
aspect_ratio = 100      # L_front/H
Ro = 0.1             # Rossby number (defines M^2)

# Derived parameters
H_front = L_front/aspect_ratio
M² = (Ro^2*f^2*L_front)/H_front
Δb = M²*L_front
κh = 1e-7
κv = 1e-7

filename = "periodic_geostrophic_adjustment"

"periodic_geostrophic_adjustment"

In [3]:

grid = RectilinearGrid(GPU(),size = (Nx, Nz), 
                       x = (-L_front/2, L_front/2),
                       z = (-H_front, 0),
                       topology = (Periodic, Flat, Bounded))

300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── Periodic x ∈ [-5000.0, 5000.0) regularly spaced with Δx=33.3333
├── Flat y                         
└── Bounded  z ∈ [-100.0, 0.0]     regularly spaced with Δz=1.25

In [4]:
horizontal_closure = HorizontalScalarDiffusivity(ν=κh, κ=κh )
vertical_closure = VerticalScalarDiffusivity(ν=κv , κ=κv )
closure = (horizontal_closure, vertical_closure)

(HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=1.0e-7), VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=1.0e-7))

In [5]:
model =  NonhydrostaticModel(; grid,
                coriolis = FPlane(f = f),
                buoyancy = BuoyancyTracer(),
                tracers = (:b,:T),
                advection = WENO(),
                closure = closure)

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×80 RectilinearGrid{Float64, Periodic, Flat, Bounded} on CUDAGPU with 3×0×3 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{3, Float64, Float32}(order=5)
├── tracers: (b, T)
├── closure: Tuple with 2 closures:
│   ├── HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=(b=1.0e-7, T=1.0e-7))
│   └── VerticalScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-7, κ=(b=1.0e-7, T=1.0e-7))
├── buoyancy: BuoyancyTracer with ĝ = NegativeZDirection()
└── coriolis: FPlane{Float64}(f=0.0001)

In [6]:
bᵢ(x, z) = Δb*sin(2*pi/L_front * x)
Tᵢ(x, z) = exp(-(x/(L_front/20)).^2)

set!(model, b= bᵢ, T= Tᵢ, u = 0, v = 0, w = 0)  # Start from rest

In [7]:
simulation = Simulation(model, Δt=20minutes, stop_time=5days)


Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 minutes
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5 days
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => 4
│   ├── stop_iteration_exceeded => -
│   ├── wall_time_limit_exceeded => e
│   └── nan_checker => }
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [8]:
conjure_time_step_wizard!(simulation, IterationInterval(20), cfl=0.2, max_Δt=20minutes)

In [9]:

wall_clock = Ref(time_ns())

function print_progress(sim)
    u, v, w = model.velocities
    progress = 100 * (time(sim) / sim.stop_time)
    elapsed = (time_ns() - wall_clock[]) / 1e9

    @printf("[%05.2f%%] i: %d, t: %s, wall time: %s, max(u): (%6.3e, %6.3e, %6.3e) m/s, next Δt: %s\n",
            progress, iteration(sim), prettytime(sim), prettytime(elapsed),
            maximum(abs, u), maximum(abs, v), maximum(abs, w), prettytime(sim.Δt))

    wall_clock[] = time_ns()

    return nothing
end

add_callback!(simulation, print_progress, IterationInterval(50))

In [10]:
# Output setup
u, v, w = model.velocities
b = model.tracers.b
wc = Field(@at (Center, Center, Center) model.velocities.w)
T = model.tracers.T

#For Lagrangian filtering
simulation.output_writers[:jld2fields] = JLD2Writer(
    model, (; b, u, v, w, wc,T), filename=filename * ".jld2", schedule=TimeInterval(1hour), overwrite_existing=true)


#for python visualisation
rm(filename * ".nc",force=true)
simulation.output_writers[:ncfields] = NetCDFWriter(
    model, (; b, u, v, w, wc,T), filename=filename * ".nc", schedule=TimeInterval(1hour), overwrite_existing=true)
    

NetCDFWriter scheduled on TimeInterval(1 hour):
├── filepath: periodic_geostrophic_adjustment.nc
├── dimensions: time(0), x_faa(300), x_caa(300), z_aaf(81), z_aac(80)
├── 6 outputs: (v, w, T, wc, b, u)
├── array_type: Array{Float32}
├── file_splitting: NoFileSplitting
└── file size: 27.5 KiB

In [11]:
@info "Running the simulation..."

run!(simulation)

@info "Simulation completed in " * prettytime(simulation.run_wall_time)

[ Info: Running the simulation...
[ Info: Initializing simulation...


[00.00%] i: 0, t: 0 seconds, wall time: 26.771 seconds, max(u): (0.000e+00, 0.000e+00, 0.000e+00) m/s, next Δt: 20 minutes


[ Info:     ... simulation initialization complete (16.332 seconds)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (2.568 seconds).


[09.40%] i: 50, t: 11.275 hours, wall time: 4.864 seconds, max(u): (2.399e-02, 5.051e-02, 4.262e-04) m/s, next Δt: 8.263 minutes
[12.50%] i: 100, t: 15 hours, wall time: 171.728 ms, max(u): (2.476e-02, 1.292e-02, 3.950e-04) m/s, next Δt: 4.097 minutes
[15.48%] i: 150, t: 18.578 hours, wall time: 190.847 ms, max(u): (1.191e-02, 2.350e-03, 2.417e-04) m/s, next Δt: 4.958 minutes
[18.54%] i: 200, t: 22.253 hours, wall time: 194.796 ms, max(u): (3.044e-02, 3.489e-02, 5.429e-04) m/s, next Δt: 3.648 minutes
[21.20%] i: 250, t: 1.060 days, wall time: 174.123 ms, max(u): (1.064e-02, 6.016e-02, 2.262e-04) m/s, next Δt: 4.414 minutes
[24.46%] i: 300, t: 1.223 days, wall time: 187.299 ms, max(u): (2.735e-02, 4.596e-02, 5.808e-04) m/s, next Δt: 4.062 minutes
[27.05%] i: 350, t: 1.352 days, wall time: 180.322 ms, max(u): (2.540e-02, 1.417e-02, 4.619e-04) m/s, next Δt: 3.917 minutes
[29.96%] i: 400, t: 1.498 days, wall time: 178.525 ms, max(u): (1.175e-02, 3.424e-03, 3.193e-04) m/s, next Δt: 5.214 mi

[ Info: Simulation is stopping after running for 26.583 seconds.
[ Info: Simulation time 5 days equals or exceeds stop time 5 days.
[ Info: Simulation completed in 26.617 seconds
